In [1]:
import pymorphy2
import string
import collections
from nltk import tokenize
from pathlib import Path
import numpy as np

In [2]:
morph = pymorphy2.MorphAnalyzer()

In [3]:
unlem_queries = [
    "Некоторые виды рыб размножаются не в воде, а на суше.",
    "Озеро Озеро — памятник природы.",
    "Канадский писатель стал первым лауреатом премии, названной в его честь."]

In [4]:
rel_2_sents = set(["Грунионы характеризуются необычным нерестовым поведением и являются одними из немногих видов рыб, которые откладывают икру не в воде, а на суше.",
               "Памятник природы занимает акваторию одноименного озера, что на левобережной пойме Десны, с прибрежной полосой и водно-болотными угодьями (внутренняя часть изгиба котловины)[2].",
               "Премия была названа в честь Грэма Гибсона, и он сам стал её первым лауреатом[4][11]."])

In [5]:
rel_1_sents = set(["С помощью гиногенеза размножаются некоторые популяции или виды рыб, представленные в природе одними самками.",
               "Статус памятника природы был присвоен решением Черниговского облисполкома от 27.04.1964 №236[1].",
               "Озеро — озеро в России, располагается в 1,4 км северо-восточнее села Юматово на территории Верхнеуслонского района Республики Татарстан[1][3].",
               "Озеро имеет округлую форму, длиной 40 м и шириной 35 м. Площадь водной поверхности озера составляет 0,08 га.",
               "Вскоре после издания «Одиннадцати канадских романистов», ставших самой известной из его книг[9], Гибсон стал одним из основателей Союза писателей Канады (англ."])

In [6]:
def get_doc_sentences(filename):
    with open(filename, "r", encoding="utf-8") as f:
        lines = f.readlines()
    sentences = []
    for line in lines:
        if line != "\n":
            sentences.extend(tokenize.sent_tokenize(line))
    return sentences

In [7]:
def get_lemmas(sentences):
    result = []
    empty_sentences_ids = []
    for i, sent in enumerate(sentences):
        sent = sent.translate(str.maketrans('', '', string.punctuation)).strip()
        splitted_sent = sent.split()
        lemmas = []
        for word in splitted_sent:
            p = morph.parse(word)[0]
            lemmas.append(p.normal_form)
        if lemmas:
            result.append(lemmas)
        else:
            empty_sentences_ids.append(i)
    return result, empty_sentences_ids

In [8]:
data_dir = Path.cwd() / "data"

docs = []
unlem_docs = []
for file in data_dir.iterdir():
    doc_sent = get_doc_sentences(data_dir / file)
    lemmas, empty_ids = get_lemmas(doc_sent)
    unlem_docs.extend([doc for i, doc in enumerate(doc_sent) if i not in empty_ids])
    docs.extend(lemmas)
    
queries, _ = get_lemmas(unlem_queries)

In [9]:
relevance = np.zeros((len(unlem_docs),), dtype=int)
for i, doc in enumerate(unlem_docs):
    if doc in rel_1_sents:
        relevance[i] = 1
    if doc in rel_2_sents:
        relevance[i] = 2

In [10]:
class Vectorizer:
    def __init__(self, docs, only_tf=False):
        self.docs = docs
        self.num_docs = len(docs)
        self.vocab = ["<unk>"] + list(set(word for doc in self.docs for word in doc))
        self.vocab_size = len(self.vocab)
        
        self.word2id = {word: idx for idx, word in enumerate(self.vocab)}
        self.only_tf = only_tf
        
    def compute_tf_idf(self):
        tf, df = np.zeros((self.num_docs, self.vocab_size), dtype=np.int), np.zeros(self.vocab_size, dtype=np.int)
        for i, doc in enumerate(self.docs):         
            v = np.zeros(self.vocab_size, dtype=np.int)
            uniq_words  = set(doc)
            for word in doc:
                v[self.word2id[word]] += 1
            
            for word in uniq_words:
                df[self.word2id[word]] += 1
            tf[i, :] = v
        df[0] = 1
        idf = np.log10(self.num_docs / df)
        # for <unk> word
        idf[0] = 0.0
        return tf, idf.reshape(1, -1)
    
    def fit_transform(self):
        self.tf, self.idf = self.compute_tf_idf()
        if self.only_tf:
            self.tf = 0.4*(self.tf != 0) + 0.6 * (self.tf / self.tf.max(axis=-1, keepdims=True))
            return self.tf * self.idf
        tf_idf_scores = self.tf * self.idf
        return tf_idf_scores
    
    def transform(self, docs):
        scores = np.zeros((len(docs), self.vocab_size), dtype=np.float) 
        for i, doc in enumerate(docs):
            for word in doc:
                word_idx = self.word2id.get(word, 0)
                if word_idx:
                    scores[i, word_idx] += 1
        if not self.only_tf:
            scores = scores * self.idf
        return scores
    
    def get_tf(self):
        return self.tf

In [11]:
vectorizer = Vectorizer(docs=docs)
docs_tfidf = vectorizer.fit_transform()
queries_tfidf = vectorizer.transform(queries)

In [12]:
docs_norm = np.linalg.norm(docs_tfidf, axis=1, keepdims=True)
queries_norm = np.linalg.norm(queries_tfidf, axis=1, keepdims=True)
dots = np.dot(docs_tfidf / docs_norm, (queries_tfidf / queries_norm).T)
dots.shape

(941, 3)

In [13]:
k = 15
top_ids = np.argpartition(dots, kth=range(-k,0), axis=0)[-k:, :][::-1, :]
col_mask = np.arange(len(queries)).reshape(1, -1).repeat(k, axis=0)
top_scores = dots[top_ids, col_mask]

In [14]:
for idx, q in enumerate(queries):
    print(f"#{idx + 1}: {unlem_queries[idx]}")
    top_docs, scores = top_ids[:, idx].tolist(), top_scores[:, idx].tolist()
    for d, s in zip(top_docs, scores):
        print(f"[{s:.3f}][rel:{relevance[d]}]: {unlem_docs[d]}")
    print("\n\n")

#1: Некоторые виды рыб размножаются не в воде, а на суше.
[0.394][rel:2]: Грунионы характеризуются необычным нерестовым поведением и являются одними из немногих видов рыб, которые откладывают икру не в воде, а на суше.
[0.370][rel:1]: С помощью гиногенеза размножаются некоторые популяции или виды рыб, представленные в природе одними самками.
[0.155][rel:0]: Виды хвостовых плавников рыб.
[0.147][rel:0]: У некоторых видов они участвуют в терморегуляции (термогенезе).
[0.144][rel:0]: У некоторых рыб могут отсутствовать брюшные, а в редких случаях — и грудные плавники.
[0.128][rel:0]: Рыбы живут в воде, которая занимает огромные пространства.
[0.127][rel:0]: Другими угрозами являются загрязнение воды, перегорождение рек, потепление, интродуцирование чужих видов, а также высыхание рек.
[0.125][rel:0]: Не характерная для рыб забота о потомстве наблюдается преимущественно у видов в приливно-отливной зоне, в узких заливах и бухтах, а также в реках и озёрах.
[0.123][rel:0]: Весь процесс занимае

In [15]:
vectorizer = Vectorizer(docs=docs, only_tf=True)
docs_tf = vectorizer.fit_transform()
queries_tf = vectorizer.transform(queries)

In [16]:
docs_norm = np.linalg.norm(docs_tf, axis=1, keepdims=True)
queries_norm = np.linalg.norm(queries_tf, axis=1, keepdims=True)
dots = np.dot(docs_tf / docs_norm, (queries_tf / queries_norm).T)

In [17]:
k = 15
top_ids = np.argpartition(dots, kth=range(-k,0), axis=0)[-k:, :][::-1, :]
col_mask = np.arange(len(queries)).reshape(1, -1).repeat(k, axis=0)
top_scores = dots[top_ids, col_mask]

In [18]:
for idx, q in enumerate(queries):
    print(f"#{idx + 1}: {unlem_queries[idx]}")
    top_docs, scores = top_ids[:, idx].tolist(), top_scores[:, idx].tolist()
    for d, s in zip(top_docs, scores):
        print(f"[{s:.3f}][rel:{relevance[d]}]: {unlem_docs[d]}")
    print("\n\n")

#1: Некоторые виды рыб размножаются не в воде, а на суше.
[0.393][rel:2]: Грунионы характеризуются необычным нерестовым поведением и являются одними из немногих видов рыб, которые откладывают икру не в воде, а на суше.
[0.327][rel:1]: С помощью гиногенеза размножаются некоторые популяции или виды рыб, представленные в природе одними самками.
[0.230][rel:0]: Виды хвостовых плавников рыб.
[0.207][rel:0]: У некоторых рыб могут отсутствовать брюшные, а в редких случаях — и грудные плавники.
[0.192][rel:0]: У некоторых видов они участвуют в терморегуляции (термогенезе).
[0.185][rel:0]: Многие виды рыб изменяют тип питания на протяжении жизни: например, в молодом возрасте питаются планктоном, а позже — рыбами или крупными беспозвоночными.
[0.184][rel:0]: Не характерная для рыб забота о потомстве наблюдается преимущественно у видов в приливно-отливной зоне, в узких заливах и бухтах, а также в реках и озёрах.
[0.174][rel:0]: Рыбы живут в воде, которая занимает огромные пространства.
[0.163][re

# Language model

In [19]:
class LanguageModel:
    def __init__(self, tf):
        self.tf = tf
        
    def p(self, term):
        return np.sum(self.tf[:, term]) / np.sum(self.tf)
    
    def p_d(self, term, document):
        return self.tf[document, term] / np.sum(self.tf[document])
    
    def get_top_k(self, query_tf, param=0.5, k=15):
        r = []
        for i in range(len(self.tf)):
            pp = 1
            for j in range(len(query_tf)):
                if query_tf[j] != 0:
                    pp *= ((1 - param) * self.p(j) + param * self.p_d(j, i))
            r.append(pp)
        idx = np.argsort(np.array(r))[-k:][::-1]
        return idx, np.array(r)[idx]

In [20]:
vectorizer = Vectorizer(docs=docs, only_tf=True)
vectorizer.fit_transform()
queries_tf = vectorizer.transform(queries)
lm = LanguageModel(vectorizer.get_tf())
queries_tf = vectorizer.transform(queries)

In [21]:
def lm_result(query_tf, param=0.5):
    indices, weights = lm.get_top_k(query_tf, param)
    for idx, w in zip(indices, weights):
        print(f"[{w}][rel:{relevance[idx]}]: {unlem_docs[idx]}")
    print("\n\n")

# Результат при lambda=0.5

In [22]:
for idx, q in enumerate(queries):
    print(f"#{idx + 1}: {unlem_queries[idx]}")
    lm_result(queries_tf[idx])

#1: Некоторые виды рыб размножаются не в воде, а на суше.
[2.764108887896997e-20][rel:2]: Грунионы характеризуются необычным нерестовым поведением и являются одними из немногих видов рыб, которые откладывают икру не в воде, а на суше.
[1.2223067778498078e-22][rel:1]: С помощью гиногенеза размножаются некоторые популяции или виды рыб, представленные в природе одними самками.
[4.138292534581941e-25][rel:0]: Не характерная для рыб забота о потомстве наблюдается преимущественно у видов в приливно-отливной зоне, в узких заливах и бухтах, а также в реках и озёрах.
[3.2197294174662787e-25][rel:0]: Другими угрозами являются загрязнение воды, перегорождение рек, потепление, интродуцирование чужих видов, а также высыхание рек.
[3.049592879574014e-25][rel:0]: Некоторые из таких рыб приспособлены к питанию планктоном, фильтруя его специализированными жаберными тычинками на жаберных дугах: так, разные виды толстолобиков (Hypophthalmichthys molitrix, Hypophthalmichthys nobilis) питаются исключительн

# Результат при lambda=0.9

In [23]:
for idx, q in enumerate(queries):
    print(f"#{idx + 1}: {unlem_queries[idx]}")
    lm_result(queries_tf[idx], param=0.9)

#1: Некоторые виды рыб размножаются не в воде, а на суше.
[3.847493040903718e-20][rel:2]: Грунионы характеризуются необычным нерестовым поведением и являются одними из немногих видов рыб, которые откладывают икру не в воде, а на суше.
[4.0591918728398873e-25][rel:1]: С помощью гиногенеза размножаются некоторые популяции или виды рыб, представленные в природе одними самками.
[2.8094550561685297e-27][rel:0]: Некоторые из таких рыб приспособлены к питанию планктоном, фильтруя его специализированными жаберными тычинками на жаберных дугах: так, разные виды толстолобиков (Hypophthalmichthys molitrix, Hypophthalmichthys nobilis) питаются исключительно за счёт этого ресурса и являются строго определёнными рыбами-фильтраторами микроскопических водорослей, которые живут в толще воды.
[1.0825946626856198e-27][rel:0]: Не характерная для рыб забота о потомстве наблюдается преимущественно у видов в приливно-отливной зоне, в узких заливах и бухтах, а также в реках и озёрах.
[5.374967738791358e-28][re

# Подсчет NDCG

In [24]:
def dcg(rel):
    return np.sum(np.array([r / np.log2(i + 1) for i, r in enumerate(rel, 1)]))

In [25]:
#1. Некоторые виды рыб размножаются не в воде, а на суше.
tf_idf_rel = [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tf_rel = [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
lm_rel_5 = [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
lm_rel_9 = [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ideal_rel = [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
idcg = dcg(ideal_rel)

In [26]:
print('NDCG')
print(f"TF-iDF: {dcg(tf_idf_rel) / idcg}") 
print(f"TF: {dcg(tf_rel) / idcg}")
print(f"LM alpha 0.5: {dcg(lm_rel_5) / idcg}")
print(f"LM alpha 0.9: {dcg(lm_rel_9) / idcg}")

NDCG
TF-iDF: 1.0
TF: 1.0
LM alpha 0.5: 1.0
LM alpha 0.9: 1.0


In [27]:
#2. Озеро Озеро — памятник природы.
tf_idf_rel = [0, 1, 1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
tf_rel = [0, 2, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
lm_rel_5 = [2, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
lm_rel_9 = [2, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
ideal_rel = [2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
idcg = dcg(ideal_rel)

In [28]:
print('NDCG')
print(f"TF-iDF: {dcg(tf_idf_rel) / idcg}") 
print(f"TF: {dcg(tf_rel) / idcg}")
print(f"LM alpha 0.5: {dcg(lm_rel_5) / idcg}")
print(f"LM alpha 0.9: {dcg(lm_rel_9) / idcg}")

NDCG
TF-iDF: 0.6479513861368325
TF: 0.6918724237169152
LM alpha 0.5: 0.8864643945764747
LM alpha 0.9: 0.8950688324185578


In [29]:
#3. Канадский писатель стал первым лауреатом премии, названной в его честь.
tf_idf_rel = [2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tf_rel = [2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
lm_rel_5 = [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
lm_rel_9 = [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ideal_rel = [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
idcg = dcg(ideal_rel)

In [30]:
print('NDCG')
print(f"TF-iDF: {dcg(tf_idf_rel) / idcg}") 
print(f"TF: {dcg(tf_rel) / idcg}")
print(f"LM alpha 0.5: {dcg(lm_rel_5) / idcg}")
print(f"LM alpha 0.9: {dcg(lm_rel_9) / idcg}")

NDCG
TF-iDF: 0.9072278740982787
TF: 0.9238850086262381
LM alpha 0.5: 1.0
LM alpha 0.9: 1.0
